## Getting WSE source data using InfluxDB client

In [1]:
#Python version 3.8.10
#Use pip install influxdb-client
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
#Token to read riverGagesSubset bucket
token = "ZmsU7WoVdvs45GVB2jNeBzDwJFjza4ZfkQple4FaRqHq8sjxDPnP5kVWOFnVygjbQPY1H_SUcXCU4xX-rn692Q=="
org = "TWI"
bucket = "riverGagesSubset"

In [2]:
client=InfluxDBClient(url="http://10.3.10.19:8086", token=token, org=org,debug=False,enable_gzip=True)
query_api = client.query_api()

In [3]:
#Define the day you need the data
#Remember the last avalaible data is yesterday at 23H
day_query="2022-09-01" #Keep this format as string

### Returning data as pandas dataframe

In [4]:
#Avoid warning about pivot because in this is not necessary that kind of format
import warnings
from influxdb_client.client.warnings import MissingPivotFunction

warnings.simplefilter("ignore", MissingPivotFunction)

In [12]:
query = f'''from(bucket: "riverGagesSubset")
  |> range(start: {day_query}T00:00:00Z, stop: {day_query}T23:59:00Z)
  |> filter(fn: (r) => r["_measurement"] == "Stage_Ft")
  |> aggregateWindow(every: 3h, fn: mean, createEmpty: false)
  |> yield(name: "mean")'''

In [13]:
data_frame = query_api.query_data_frame(org=org, query=query)
data_frame = data_frame.drop(columns=["_start","result","table","_field","_stop","_measurement"])
data_frame = data_frame.rename(columns={"_value": "mean_value_adjusted_mts", "_time": "Datetime"})
data_frame

,Datetime,mean_value_adjusted_mts,Latitude,Longitude,Station,river_mile
0,2022-09-01 03:00:00+00:00,0.539496,28.93231,-89.4071,rg_01670,17.9
1,2022-09-01 06:00:00+00:00,0.579120,28.93231,-89.4071,rg_01670,17.9
2,2022-09-01 09:00:00+00:00,0.588264,28.93231,-89.4071,rg_01670,17.9
3,2022-09-01 12:00:00+00:00,0.508000,28.93231,-89.4071,rg_01670,17.9
4,2022-09-01 15:00:00+00:00,0.353568,28.93231,-89.4071,rg_01670,17.9
...,...,...,...,...,...,...
291,2022-09-01 12:00:00+00:00,95.873824,37.30415,-89.5175,rg_CE401278,52.1
292,2022-09-01 15:00:00+00:00,95.880936,37.30415,-89.5175,rg_CE401278,52.1
293,2022-09-01 18:00:00+00:00,95.879920,37.30415,-89.5175,rg_CE401278,52.1
294,2022-09-01 21:00:00+00:00,95.876872,37.30415,-89.5175,rg_CE401278,52.1
